In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import os
import datetime as dt
import xgboost as xgb
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import catboost
import sklearn
from sklearn.model_selection import cross_val_score, KFold, learning_curve, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import operator
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import seaborn as sns
import datetime
from time import strftime, gmtime, time
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

tqdm.pandas()

In [3]:
sns.set_context("poster") 
plt.style.use('seaborn-poster')

In [4]:
# !pip install xgboost
# !pip install sklearn

In [5]:
def feature_importances(model) -> pd.DataFrame:

    importance_dict = {}

    for feature, importance in zip(X_train.columns, model.feature_importances_):
        importance_dict[feature] = importance

    df_importance = pd.DataFrame(sorted(importance_dict.items(), key=operator.itemgetter(1)))
    df_importance = df_importance.set_index(0).sort_values(by=[1], ascending=False)

    return df_importance

In [52]:
cat_tester = pd.read_csv("Parking_added_tester.csv")
# cat_tester_1 = pd.read_csv("All_data_df.csv")
cat_tester_1 = pd.read_csv("Complete dataframe/data_preprocessing.csv")
cat_tester_1

cat_tester_1 = cat_tester_1[cat_tester_1['Datetime'] > '2021']
cat_tester_1
# park specifier
# cat_tester_1 = cat_tester_1[cat_tester_1['Location']== 'Oosterpark']


,Datetime,Location,Visits,Date,Time,Journeys,Windspeed,Temperature,Clouds,Rain amount,...,Holiday_Name_Summer holiday,Holiday_Name_Whit,Holiday_Name_Winter holiday,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index
87323,2021-01-01 00:00:00,Amstelpark,25.0,2021-01-01,00:00:00,21.818182,2.0,-0.4,1.00,0.0,...,0,0,1,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87324,2021-01-01 00:00:00,Beatrixpark,154.0,2021-01-01,00:00:00,88.000000,2.0,-0.4,1.00,0.0,...,0,0,1,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87325,2021-01-01 00:00:00,Erasmuspark,323.0,2021-01-01,00:00:00,13.933333,2.0,-0.4,1.00,0.0,...,0,0,1,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87326,2021-01-01 00:00:00,Flevopark,65.0,2021-01-01,00:00:00,61.777778,2.0,-0.4,1.00,0.0,...,0,0,1,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87327,2021-01-01 00:00:00,Gaasperpark,25.0,2021-01-01,00:00:00,15.400000,2.0,-0.4,1.00,0.0,...,0,0,1,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724612,2022-01-06 14:45:00,Westerpark West,91.0,2022-01-06,14:45:00,10.000000,5.0,5.4,3.25,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724613,2022-01-06 15:00:00,Flevopark,165.0,2022-01-06,15:00:00,18.870370,5.0,5.2,4.00,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724614,2022-01-06 15:00:00,Sarphatipark,776.0,2022-01-06,15:00:00,115.074419,5.0,5.2,4.00,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724615,2022-01-06 15:00:00,Sloterpark,366.0,2022-01-06,15:00:00,23.804651,5.0,5.2,4.00,0.0,...,0,0,1,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44


In [53]:
# X, y = cat_tester.loc[:, cat_tester.columns != "Visits"], cat_tester[["Visits"]]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# cat_features = ['Date_x','Time','Location']

# X_train = X_train[['Location', 'Date_x', 'Time', 'Journeys' ,'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday','capacity.filled']]
# X_test = X_test[['Location', 'Date_x', 'Time', 'Journeys', 'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday','capacity.filled']]

# assert "ARRIVAL_DELAY" not in X_train.columns
# assert y_train.shape[1] == 1

In [54]:
# X, y = cat_tester_1.loc[:, cat_tester_1.columns != "Visits"], cat_tester_1[["Visits"]]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# cat_features = ['Date','Time','Location']

# X_train = X_train[['Location', 'Date', 'Time', 'Journeys' ,'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
# X_test = X_test[['Location', 'Date', 'Time', 'Journeys', 'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]

# assert "ARRIVAL_DELAY" not in X_train.columns
# assert y_train.shape[1] == 1

In [56]:
X, y = cat_tester_1.loc[:, cat_tester_1.columns != "Visits"], cat_tester_1[["Visits"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
cat_features = ['Datetime','Date','Time','Location']

X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]

assert "ARRIVAL_DELAY" not in X_train.columns
assert y_train.shape[1] == 1

In [59]:
locations = cat_tester_1.Location.unique()
# for x in locations:
park_name = []
Iterations = []
RMSE_Train_Scores = []
MAE_Train_scores = []
RMSE_Test_Scores = []
MAE_Test_Scores = []
Run_time = []

for x in locations:
    test = cat_tester_1[cat_tester_1['Location']== x]
    
    X, y = test.loc[:, test.columns != "Visits"], test[["Visits"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    cat_features = ['Datetime','Date','Time','Location']

    X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    assert "ARRIVAL_DELAY" not in X_train.columns
    assert y_train.shape[1] == 1
    
    cat_scores = {}
    
    for iterations in range(25, 1050, 1000):
    # for iterations in range(25, 500, 50):

#         print(f"Fitting model for {iterations} iterations...")

        start_time = time()

        cat_scores[iterations] = {}

        cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)

        cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


        cat_scores[iterations]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
        pred = cat_model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred) 
        cat_scores[iterations]['MAE Train'] = mae_train
        
        cat_scores[iterations]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
        pred = cat_model.predict(X_test)
        mae_val = mean_absolute_error(y_test, pred)
        cat_scores[iterations]['MAE Test'] = mae_val

        cat_scores[iterations]["time"] = time() - start_time
        
        park_name.append(x)
        Iterations.append(iterations)
        RMSE_Train_Scores.append(cat_model.get_best_score()['learn'])
        MAE_Train_scores.append(mae_train)
        RMSE_Test_Scores.append(cat_model.get_best_score()['validation'])
        MAE_Test_Scores.append(mae_val)
        Run_time.append(time() - start_time)    
        
    print(f"Scores for {x}")
#         print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")
    print(cat_scores)

Scores for Amstelpark
{25: {'RMSE train': 42.681041628993995, 'MAE Train': 24.67880871444061, 'RMSE test': 44.53619757586476, 'MAE Test': 25.157814874974402, 'time': 0.22666692733764648}, 1025: {'RMSE train': 27.652341519835083, 'MAE Train': 17.157167214770936, 'RMSE test': 35.76119162484711, 'MAE Test': 19.631670647867434, 'time': 14.676554918289185}}
Scores for Beatrixpark
{25: {'RMSE train': 179.39229307652332, 'MAE Train': 114.05718492098451, 'RMSE test': 188.52215833013008, 'MAE Test': 120.71823559164207, 'time': 0.2057037353515625}, 1025: {'RMSE train': 133.63433106635338, 'MAE Train': 84.75165841495904, 'RMSE test': 159.00978144844356, 'MAE Test': 97.41139439995065, 'time': 14.911749124526978}}
Scores for Erasmuspark
{25: {'RMSE train': 81.27470812536339, 'MAE Train': 45.19151479158247, 'RMSE test': 79.83926829094725, 'MAE Test': 46.79555098025624, 'time': 0.2349710464477539}, 1025: {'RMSE train': 55.83804479172042, 'MAE Train': 33.223053251581625, 'RMSE test': 62.34059775153295

In [11]:
# cat_scores

In [12]:
# cat_model.get_best_score()

In [104]:
feature_importances(cat_model)

,1
0,
Time,16.574282
Journeys,15.331945
Date,11.235907
Temperature,8.308122
Windspeed,6.678763
workplaces_percent_change_from_baseline,5.484522
Day_x,4.719563
transit_stations_percent_change_from_baseline,3.466249
parks_percent_change_from_baseline,3.386816


In [61]:
henk =pd.DataFrame(list(zip(park_name,Iterations, RMSE_Train_Scores, MAE_Train_scores, RMSE_Test_Scores, MAE_Test_Scores, Run_time)), 
                   columns=['Park_name','Iterations','RMSE_Train','MAE_Train', 'RMSE_Test','MAE_Test', 'Run_Time'])
for x in range(len(henk)):
    henk['RMSE_Train'][x] = henk['RMSE_Train'][x]['RMSE']
    henk['RMSE_Test'][x] = henk['RMSE_Test'][x]['RMSE']
henk = henk[henk['Iterations']==1025]
henk

/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_30600/2460751921.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  henk['RMSE_Train'][x] = henk['RMSE_Train'][x]['RMSE']
/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_30600/2460751921.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  henk['RMSE_Test'][x] = henk['RMSE_Test'][x]['RMSE']


,Park_name,Iterations,RMSE_Train,MAE_Train,RMSE_Test,MAE_Test,Run_Time
1,Amstelpark,1025,27.652342,17.157167,35.761192,19.631671,14.676566
3,Beatrixpark,1025,133.634331,84.751658,159.009781,97.411394,14.911762
5,Erasmuspark,1025,55.838045,33.223053,62.340598,37.089305,16.075742
7,Flevopark,1025,27.533174,18.316545,30.908198,20.391002,16.563380
9,Gaasperpark,1025,25.055268,16.407402,28.493102,18.583203,16.626602
11,Nelson Mandelapark,1025,43.655091,27.801391,50.028936,31.195619,16.797015
13,Noorderpark,1025,117.212609,74.559135,135.783588,84.037700,16.651803
15,Oosterpark,1025,125.598849,79.950404,150.134196,89.801522,17.804646
17,Park Frankendael,1025,57.913041,38.498155,62.836992,41.556702,16.562274
19,Sarphatipark,1025,98.215255,61.490111,110.01926,68.172282,16.710261


In [18]:
henk['RMSE_Test'].mean()

nan

In [206]:
henk['MAE_Test'].mean()

45.623617516218225

In [62]:
cat_model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'RMSE',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1025,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': True,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0

In [38]:
cat_model.get_feature_importance()

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

array([0.00000000e+00, 0.00000000e+00, 1.10158292e+01, 1.82196914e+01,
       1.84145233e+01, 4.67533912e+00, 8.56203894e+00, 2.56720425e+00,
       9.49477613e-01, 5.55437742e-01, 2.17277408e+00, 9.13002392e-02,
       1.74424823e-01, 1.65649141e-03, 1.43954224e-01, 6.94433367e-03,
       0.00000000e+00, 0.00000000e+00, 2.68100509e+00, 2.72883988e+00,
       0.00000000e+00, 2.65363154e+00, 2.78448902e+00, 4.22376143e+00,
       5.21039661e+00, 6.15618258e+00, 2.73540705e+00, 3.27569100e+00])

In [39]:
cat_model.get_params()

{'iterations': 1025,
 'loss_function': 'RMSE',
 'verbose': False,
 'cat_features': ['Datetime', 'Date', 'Time', 'Location']}

In [40]:
cat_model.get_scale_and_bias()

(1.0, 94.81302642822266)

In [69]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [89]:
from catboost import CatBoost
import numpy as np
locations = cat_tester_1.Location.unique()
# for x in locations:
park_name = []
Iterations = []
RMSE_Train_Scores = []
MAE_Train_scores = []
RMSE_Test_Scores = []
MAE_Test_Scores = []
Run_time = []
cat_scores = {}

for x in locations:
    test = cat_tester_1[cat_tester_1['Location']== x]
    
    X, y = test.loc[:, test.columns != "Visits"], test[["Visits"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    cat_features = ['Datetime','Date','Time','Location']

    X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    assert "ARRIVAL_DELAY" not in X_train.columns
    assert y_train.shape[1] == 1
    
    cat_model = catboost.CatBoostRegressor(cat_features=cat_features,verbose=False)
    
    grid = {'learning_rate': [0.1, 0.25],
        'depth': [10,12],
        'l2_leaf_reg': [3, 5, 7, 9],
        'iterations':[350]}

    grid_search_result = cat_model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       plot=True)
    print('Optimal parameters for', x)
    print(grid_search_result['params'])


    
#     for iterations in range(25, 1050, 1000):
    # for iterations in range(25, 500, 50):

#         print(f"Fitting model for {iterations} iterations...")

#     start_time = time()

#     cat_scores[iterations] = {}

#     cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)

#     cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


#     cat_scores[iterations]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
#     pred = cat_model.predict(X_train)
#     mae_train = mean_absolute_error(y_train, pred) 
#     cat_scores[iterations]['MAE Train'] = mae_train

#     cat_scores[iterations]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
#     pred = cat_model.predict(X_test)
#     mae_val = mean_absolute_error(y_test, pred)
#     cat_scores[iterations]['MAE Test'] = mae_val

#     cat_scores[iterations]["time"] = time() - start_time

#     park_name.append(x)
#     Iterations.append(iterations)
#     RMSE_Train_Scores.append(cat_model.get_best_score()['learn'])
#     MAE_Train_scores.append(mae_train)
#     RMSE_Test_Scores.append(cat_model.get_best_score()['validation'])
#     MAE_Test_Scores.append(mae_val)
#     Run_time.append(time() - start_time)    
        
#     print(f"Scores for {x}")
# #         print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")
#     print(cat_scores)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 35.44269508
bestIteration = 349

0:	loss: 35.4426951	best: 35.4426951 (0)	total: 10s	remaining: 2m 30s

bestTest = 34.66765103
bestIteration = 337

1:	loss: 34.6676510	best: 34.6676510 (1)	total: 20.5s	remaining: 2m 23s

bestTest = 35.65980375
bestIteration = 346

2:	loss: 35.6598038	best: 34.6676510 (1)	total: 30.2s	remaining: 2m 10s

bestTest = 35.13564135
bestIteration = 341

3:	loss: 35.1356414	best: 34.6676510 (1)	total: 41s	remaining: 2m 3s

bestTest = 36.16642975
bestIteration = 349

4:	loss: 36.1664297	best: 34.6676510 (1)	total: 51.6s	remaining: 1m 53s

bestTest = 35.16947181
bestIteration = 349

5:	loss: 35.1694718	best: 34.6676510 (1)	total: 1m 1s	remaining: 1m 42s

bestTest = 35.91086344
bestIteration = 348

6:	loss: 35.9108634	best: 34.6676510 (1)	total: 1m 11s	remaining: 1m 31s

bestTest = 35.28191251
bestIteration = 349

7:	loss: 35.2819125	best: 34.6676510 (1)	total: 1m 20s	remaining: 1m 20s

bestTest = 34.75347841
bestIteration = 349

8:	loss: 34.7534784	be

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 156.363839
bestIteration = 349

0:	loss: 156.3638390	best: 156.3638390 (0)	total: 10.6s	remaining: 2m 38s

bestTest = 154.6522748
bestIteration = 342

1:	loss: 154.6522748	best: 154.6522748 (1)	total: 22.4s	remaining: 2m 37s

bestTest = 157.2585378
bestIteration = 348

2:	loss: 157.2585378	best: 154.6522748 (1)	total: 34.6s	remaining: 2m 29s

bestTest = 154.1510249
bestIteration = 349

3:	loss: 154.1510249	best: 154.1510249 (3)	total: 45.8s	remaining: 2m 17s

bestTest = 160.2539598
bestIteration = 349

4:	loss: 160.2539598	best: 154.1510249 (3)	total: 56.8s	remaining: 2m 4s

bestTest = 155.7776162
bestIteration = 349

5:	loss: 155.7776162	best: 154.1510249 (3)	total: 1m 8s	remaining: 1m 53s

bestTest = 159.3302766
bestIteration = 349

6:	loss: 159.3302766	best: 154.1510249 (3)	total: 1m 19s	remaining: 1m 42s

bestTest = 155.2829549
bestIteration = 349

7:	loss: 155.2829549	best: 154.1510249 (3)	total: 1m 31s	remaining: 1m 31s

bestTest = 154.2872085
bestIteration = 349

8:	

...


bestTest = 31.3989306
bestIteration = 349

0:	loss: 31.3989306	best: 31.3989306 (0)	total: 8.07s	remaining: 2m 1s

bestTest = 31.29290565
bestIteration = 349

1:	loss: 31.2929056	best: 31.2929056 (1)	total: 16.5s	remaining: 1m 55s

bestTest = 31.71276751
bestIteration = 349

2:	loss: 31.7127675	best: 31.2929056 (1)	total: 24.8s	remaining: 1m 47s

bestTest = 31.71138905
bestIteration = 347

3:	loss: 31.7113891	best: 31.2929056 (1)	total: 33.1s	remaining: 1m 39s

bestTest = 32.08636291
bestIteration = 349

4:	loss: 32.0863629	best: 31.2929056 (1)	total: 41.5s	remaining: 1m 31s

bestTest = 31.60986231
bestIteration = 349

5:	loss: 31.6098623	best: 31.2929056 (1)	total: 50.1s	remaining: 1m 23s

bestTest = 32.19582019
bestIteration = 347

6:	loss: 32.1958202	best: 31.2929056 (1)	total: 58.3s	remaining: 1m 14s

bestTest = 30.71679948
bestIteration = 349

7:	loss: 30.7167995	best: 30.7167995 (7)	total: 1m 7s	remaining: 1m 7s

bestTest = 31.29516042
bestIteration = 349

8:	loss: 31.2951604	be

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 35.95800026
bestIteration = 348

0:	loss: 35.9580003	best: 35.9580003 (0)	total: 8.19s	remaining: 2m 2s

bestTest = 35.02882172
bestIteration = 340

1:	loss: 35.0288217	best: 35.0288217 (1)	total: 16.5s	remaining: 1m 55s

bestTest = 36.0512583
bestIteration = 349

2:	loss: 36.0512583	best: 35.0288217 (1)	total: 24.6s	remaining: 1m 46s

bestTest = 35.30238578
bestIteration = 345

3:	loss: 35.3023858	best: 35.0288217 (1)	total: 33.5s	remaining: 1m 40s

bestTest = 36.46160475
bestIteration = 349

4:	loss: 36.4616048	best: 35.0288217 (1)	total: 42.2s	remaining: 1m 32s

bestTest = 35.34597151
bestIteration = 349

5:	loss: 35.3459715	best: 35.0288217 (1)	total: 51.1s	remaining: 1m 25s

bestTest = 36.36405368
bestIteration = 349

6:	loss: 36.3640537	best: 35.0288217 (1)	total: 59.4s	remaining: 1m 16s

bestTest = 35.32533775
bestIteration = 346

7:	loss: 35.3253378	best: 35.0288217 (1)	total: 1m 7s	remaining: 1m 7s

bestTest = 35.62375457
bestIteration = 349

8:	loss: 35.6237546	be

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 35.70073699
bestIteration = 349

0:	loss: 35.7007370	best: 35.7007370 (0)	total: 9.48s	remaining: 2m 22s

bestTest = 35.75310775
bestIteration = 349

1:	loss: 35.7531077	best: 35.7007370 (0)	total: 18.3s	remaining: 2m 8s

bestTest = 36.34513131
bestIteration = 349

2:	loss: 36.3451313	best: 35.7007370 (0)	total: 26.9s	remaining: 1m 56s

bestTest = 35.46219977
bestIteration = 347

3:	loss: 35.4621998	best: 35.4621998 (3)	total: 35.7s	remaining: 1m 47s

bestTest = 36.27183441
bestIteration = 349

4:	loss: 36.2718344	best: 35.4621998 (3)	total: 44.8s	remaining: 1m 38s

bestTest = 35.20007985
bestIteration = 349

5:	loss: 35.2000798	best: 35.2000798 (5)	total: 53.7s	remaining: 1m 29s

bestTest = 36.87082513
bestIteration = 349

6:	loss: 36.8708251	best: 35.2000798 (5)	total: 1m 2s	remaining: 1m 20s

bestTest = 35.4892442
bestIteration = 349

7:	loss: 35.4892442	best: 35.2000798 (5)	total: 1m 10s	remaining: 1m 10s

bestTest = 35.85219429
bestIteration = 349

8:	loss: 35.8521943	

In [87]:
# for saving the model
# cat_model.save_model(fname, )

Amstelpark


{'depth': 10, 'l2_leaf_reg': 9, 'iterations': 350, 'learning_rate': 0.5}

In [36]:
cat_model.select_features(
                X,
                y=None,
                eval_set=None,
                features_for_select=None,
                num_features_to_select=None,
                algorithm=None,
                steps=None,
                shap_calc_type=None,
                train_final_model=False,
                verbose=None,
                logging_level=None,
                plot=False,
                log_cout=sys.stdout,
                log_cerr=sys.stderr)

# https://catboost.ai/en/docs/concepts/python-reference_catboost_select_features

NameError: name 'sys' is not defined

In [90]:
Optimal_parameters = {
    'Amstelpark': {'depth': 12, 'l2_leaf_reg': 3,'learning_rate': 0.25},
    'Beatrixpark': {'depth': 12, 'l2_leaf_reg': 7, 'learning_rate': 0.25},
    'Erasmuspark': {'depth': 12, 'l2_leaf_reg': 9, 'learning_rate': 0.25},
    'Flevopark': {'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'Gaasperpark': {'depth': 10, 'l2_leaf_reg': 5,'learning_rate': 0.25},
    'Nelson Mandelapark': {'depth': 10, 'l2_leaf_reg': 5, 'learning_rate': 0.25},
    'Noorderpark': {'depth': 12, 'l2_leaf_reg': 5, 'learning_rate': 0.25},
    'Oosterpark': {'depth': 12, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'Park Frankendael': {'depth': 12, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'Sarphatipark': {'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'Sloterpark': {'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'WH Vliegenbos': {'depth': 12, 'l2_leaf_reg': 5, 'learning_rate': 0.25},
    'Vondelpark West': {'depth': 12, 'l2_leaf_reg': 5, 'learning_rate': 0.25},
    'Westergasfabriek': {'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.25},
    'Vondelpark Oost 1': {'depth': 12, 'l2_leaf_reg': 9, 'learning_rate': 0.25},
    'Vondelpark Oost 2':{'depth': 12, 'l2_leaf_reg': 7, 'learning_rate': 0.25},
    'Vondelpark Oost 3':{'depth': 10, 'l2_leaf_reg': 5, 'learning_rate': 0.25},
    'Westerpark Centrum':{'depth': 10, 'l2_leaf_reg': 9, 'learning_rate': 0.25},
    'Westerpark Oost':{'depth': 12, 'l2_leaf_reg': 9, 'learning_rate': 0.25},
    'Westerpark West':{'depth': 10, 'l2_leaf_reg': 7, 'learning_rate': 0.25},
}

In [102]:
from catboost import CatBoost
import numpy as np
locations = cat_tester_1.Location.unique()
# for x in locations:
park_name = []
Iterations = []
RMSE_Train_Scores = []
MAE_Train_scores = []
RMSE_Test_Scores = []
MAE_Test_Scores = []
Run_time = []
cat_scores = {}

for x in locations:
    test = cat_tester_1[cat_tester_1['Location']== x]
    
    X, y = test.loc[:, test.columns != "Visits"], test[["Visits"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    cat_features = ['Datetime','Date','Time','Location']

    X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index','Holiday_Name_Ascension Thursday',
       'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
       'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
       'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
       'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
       'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
    assert "ARRIVAL_DELAY" not in X_train.columns
    assert y_train.shape[1] == 1
    
    iteration = 2000
    
    cat_model = catboost.CatBoostRegressor(cat_features=cat_features,verbose=False, iterations= iteration ,depth = Optimal_parameters[x]['depth'], l2_leaf_reg = Optimal_parameters[x]['l2_leaf_reg'], learning_rate= Optimal_parameters[x]['learning_rate'])

        
#     grid = {'learning_rate': [0.1, 0.25],
#         'depth': [10,12],
#         'l2_leaf_reg': [3, 5, 7, 9],
#         'iterations':[350]}

#     grid_search_result = cat_model.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                        plot=True)
#     print('Optimal parameters for', x)
#     print(grid_search_result['params'])


    
#     for iterations in range(25, 1050, 1000):
    # for iterations in range(25, 500, 50):

#         print(f"Fitting model for {iterations} iterations...")

    start_time = time()

    cat_scores[iteration] = {}


    cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


    cat_scores[iteration]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
    pred = cat_model.predict(X_train)
    mae_train = mean_absolute_error(y_train, pred) 
    cat_scores[iteration]['MAE Train'] = mae_train

    cat_scores[iteration]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
    pred = cat_model.predict(X_test)
    mae_val = mean_absolute_error(y_test, pred)
    cat_scores[iteration]['MAE Test'] = mae_val

    cat_scores[iteration]["time"] = time() - start_time

    park_name.append(x)
    Iterations.append(iteration)
    RMSE_Train_Scores.append(cat_model.get_best_score()['learn'])
    MAE_Train_scores.append(mae_train)
    RMSE_Test_Scores.append(cat_model.get_best_score()['validation'])
    MAE_Test_Scores.append(mae_val)
    Run_time.append(time() - start_time)    
        
    print(f"Scores for {x}")
#         print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")
    print(cat_scores)

Scores for Amstelpark
{2000: {'RMSE train': 0.920159848996134, 'MAE Train': 9.446752528495937, 'RMSE test': 32.10595605604654, 'MAE Test': 16.30147918346498, 'time': 133.814129114151}}
Scores for Beatrixpark
{2000: {'RMSE train': 8.030897759710092, 'MAE Train': 48.48048938958235, 'RMSE test': 140.67280387096463, 'MAE Test': 81.31001809214547, 'time': 134.68694686889648}}
Scores for Erasmuspark
{2000: {'RMSE train': 4.545787586613939, 'MAE Train': 19.83568087539801, 'RMSE test': 56.14830454692462, 'MAE Test': 31.526172791945772, 'time': 134.59458422660828}}
Scores for Flevopark
{2000: {'RMSE train': 3.7395606602954006, 'MAE Train': 10.324629679351887, 'RMSE test': 27.392184801716866, 'MAE Test': 17.253786201686918, 'time': 67.8850519657135}}
Scores for Gaasperpark
{2000: {'RMSE train': 3.592777190024443, 'MAE Train': 9.256306267897791, 'RMSE test': 24.75595485380488, 'MAE Test': 15.629309841615534, 'time': 61.46732783317566}}
Scores for Nelson Mandelapark
{2000: {'RMSE train': 6.9673902

In [103]:
results =pd.DataFrame(list(zip(park_name,Iterations, RMSE_Train_Scores, MAE_Train_scores, RMSE_Test_Scores, MAE_Test_Scores, Run_time)), 
                   columns=['Park_name','Iterations','RMSE_Train','MAE_Train', 'RMSE_Test','MAE_Test', 'Run_Time'])
for x in range(len(results)):
    results['RMSE_Train'][x] = results['RMSE_Train'][x]['RMSE']
    results['RMSE_Test'][x] = results['RMSE_Test'][x]['RMSE']
results

/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_30600/1039197985.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['RMSE_Train'][x] = results['RMSE_Train'][x]['RMSE']
/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_30600/1039197985.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['RMSE_Test'][x] = results['RMSE_Test'][x]['RMSE']


,Park_name,Iterations,RMSE_Train,MAE_Train,RMSE_Test,MAE_Test,Run_Time
0,Amstelpark,2000,0.92016,9.446753,32.105956,16.301479,133.814159
1,Beatrixpark,2000,8.030898,48.480489,140.672804,81.310018,134.686960
2,Erasmuspark,2000,4.545788,19.835681,56.148305,31.526173,134.594595
3,Flevopark,2000,3.739561,10.324630,27.392185,17.253786,67.885114
4,Gaasperpark,2000,3.592777,9.256306,24.755955,15.629310,61.467339
5,Nelson Mandelapark,2000,6.96739,16.179430,43.712227,26.078031,64.672678
6,Noorderpark,2000,5.850771,40.732603,124.234077,71.458775,150.892764
7,Oosterpark,2000,4.922153,42.875023,132.819447,73.670052,143.430952
8,Park Frankendael,2000,2.696456,21.505937,55.064611,35.712409,148.551696
9,Sarphatipark,2000,12.988056,35.746597,99.508438,57.137331,69.276169
